In [3]:
%%writefile listings.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>SYOH – Browse Listings</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <style>
    :root{
      --bg:#020617; --bg-soft:#0f172a; --accent:#22c55e; --accent-soft:#16a34a;
      --text:#e5e7eb; --muted:#9ca3af; --border:rgba(148,163,184,0.3);
    }
    *{box-sizing:border-box}
    body{
      margin:0; font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",sans-serif;
      background: radial-gradient(circle at top, #0b1120, #020617 55%);
      color:var(--text);
    }
    header{
      position:fixed; top:0; left:0; right:0;
      padding:.75rem 1.5rem;
      display:flex; justify-content:space-between; align-items:center;
      backdrop-filter: blur(16px);
      background: rgba(15,23,42,0.9);
      border-bottom:1px solid var(--border);
      z-index:50;
    }
    .logo{font-weight:700; letter-spacing:.08em; text-transform:uppercase; font-size:.9rem;}
    nav a{margin-left:1rem; font-size:.9rem; color:var(--muted); text-decoration:none}
    nav a:hover{color:var(--accent)}
    main{
      padding-top:4.5rem;
      padding-inline:1.5rem;
      max-width:1100px;
      margin:0 auto;
    }
    h1{font-size:clamp(2rem,4vw,2.6rem); margin:.5rem 0 .5rem}
    .lead{color:var(--muted); max-width:720px; margin:0 0 1.25rem}
    .bar{
      display:grid;
      grid-template-columns: 1fr 180px 140px 140px;
      gap:.75rem;
      margin:1rem 0 1.5rem;
    }
    @media (max-width:900px){ .bar{grid-template-columns:1fr 1fr} }
    @media (max-width:520px){ .bar{grid-template-columns:1fr} }
    input, select{
      width:100%;
      background: rgba(2,6,23,.55);
      border:1px solid var(--border);
      border-radius: .9rem;
      color:var(--text);
      padding:.75rem .9rem;
      outline:none;
      font-size:.95rem;
    }
    .btn{
      border-radius:9999px; padding:.8rem 1.1rem; border:1px solid transparent;
      font-weight:600; cursor:pointer; font-size:.92rem; text-decoration:none;
      display:inline-flex; align-items:center; justify-content:center;
    }
    .btn-primary{
      background: linear-gradient(135deg, var(--accent), var(--accent-soft));
      color:#022c22;
      box-shadow: 0 18px 40px rgba(34,197,94,0.25);
    }
    .btn-ghost{background:transparent; color:var(--text); border-color:var(--border)}
    .btn-ghost:hover{border-color:var(--accent)}
    .grid{
      display:grid;
      grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
      gap:1.25rem;
    }
    .card{
      background: var(--bg-soft);
      border:1px solid var(--border);
      border-radius:1rem;
      padding:1.25rem;
    }
    .pill{
      display:inline-block;
      padding:.32rem .65rem;
      border-radius:999px;
      border:1px solid var(--border);
      color:var(--muted);
      font-size:.78rem;
      margin-bottom:.6rem;
    }
    .title{font-weight:700; font-size:1.05rem; margin:.2rem 0}
    .city{color:var(--muted); font-size:.92rem; margin-bottom:.6rem}
    .price{color:var(--accent); font-weight:800; font-size:1.25rem; margin:.15rem 0}
    .meta{color:var(--muted); font-size:.88rem; margin:.2rem 0 .9rem}
    .actions{display:flex; gap:.6rem; flex-wrap:wrap}
    footer{
      margin-top:3rem;
      padding:1.5rem;
      text-align:center;
      color:var(--muted);
      border-top:1px solid var(--border);
      font-size:.85rem;
    }
  </style>
</head>
<body>

  <header>
    <div class="logo">SYOH</div>
    <nav>
      <a href="index.html">Home</a>
      <a href="create-listing.html">Create Listing</a>
      <a href="listings.html">Browse Listings</a>
    </nav>
  </header>

  <main>
    <h1>Browse Listings</h1>
    <p class="lead">
      Demo listing marketplace for SYOH. Use filters below (works fully client-side).
    </p>

    <div class="bar">
      <input id="q" placeholder="Search city or title (e.g., Austin)" />
      <input id="maxPrice" type="number" placeholder="Max price" />
      <select id="beds">
        <option value="">Beds+</option>
        <option value="1">1+</option>
        <option value="2">2+</option>
        <option value="3">3+</option>
        <option value="4">4+</option>
        <option value="5">5+</option>
      </select>
      <button class="btn btn-primary" id="apply">Apply</button>
    </div>

    <div class="grid" id="cards"></div>
  </main>

  <footer>
    © <span id="year"></span> SYOH – Sell Your Own Home
  </footer>

  <script>
    document.getElementById("year").textContent = new Date().getFullYear();

    const DATA = [
      {tag:"New", title:"Modern Townhome", city:"Austin, TX", price:485000, beds:3, baths:2.5, sqft:1680},
      {tag:"Hot", title:"Single Family Home", city:"Plano, TX", price:620000, beds:4, baths:3, sqft:2650},
      {tag:"Featured", title:"Downtown Condo", city:"Chicago, IL", price:399000, beds:2, baths:2, sqft:1325},
      {tag:"Value", title:"Suburban Starter Home", city:"Tulsa, OK", price:310000, beds:3, baths:2, sqft:1410},
      {tag:"Pro", title:"Spacious Family Home", city:"Frisco, TX", price:640000, beds:4, baths:3, sqft:2700},
      {tag:"New", title:"Charming Bungalow", city:"Minneapolis, MN", price:355000, beds:2, baths:1, sqft:1180}
    ];

    function money(n){
      return new Intl.NumberFormat("en-US",{style:"currency",currency:"USD",maximumFractionDigits:0}).format(n);
    }

    function card(x){
      return `
        <div class="card">
          <div class="pill">${x.tag}</div>
          <div class="title">${x.title}</div>
          <div class="city">${x.city}</div>
          <div class="price">${money(x.price)}</div>
          <div class="meta">${x.beds} bd • ${x.baths} ba • ${x.sqft} sqft</div>
          <div class="actions">
            <a class="btn btn-primary" href="create-listing.html">Contact Seller</a>
            <a class="btn btn-ghost" href="index.html#how">How SYOH works</a>
          </div>
        </div>
      `;
    }

    function render(list){
      document.getElementById("cards").innerHTML = list.map(card).join("");
    }

    function apply(){
      const q = (document.getElementById("q").value || "").toLowerCase().trim();
      const maxPrice = Number(document.getElementById("maxPrice").value || 0);
      const beds = Number(document.getElementById("beds").value || 0);

      let out = [...DATA];

      if(q){
        out = out.filter(x =>
          x.city.toLowerCase().includes(q) ||
          x.title.toLowerCase().includes(q)
        );
      }
      if(maxPrice > 0){
        out = out.filter(x => x.price <= maxPrice);
      }
      if(beds > 0){
        out = out.filter(x => x.beds >= beds);
      }

      render(out);
    }

    document.getElementById("apply").addEventListener("click", apply);
    render(DATA);
  </script>
</body>
</html>


Writing listings.html


In [4]:
with open('listings.html', 'r') as f:
    html_content = f.read()
print(html_content)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>SYOH – Browse Listings</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <style>
    :root{
      --bg:#020617; --bg-soft:#0f172a; --accent:#22c55e; --accent-soft:#16a34a;
      --text:#e5e7eb; --muted:#9ca3af; --border:rgba(148,163,184,0.3);
    }
    *{box-sizing:border-box}
    body{
      margin:0; font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",sans-serif;
      background: radial-gradient(circle at top, #0b1120, #020617 55%);
      color:var(--text);
    }
    header{
      position:fixed; top:0; left:0; right:0;
      padding:.75rem 1.5rem;
      display:flex; justify-content:space-between; align-items:center;
      backdrop-filter: blur(16px);
      background: rgba(15,23,42,0.9);
      border-bottom:1px solid var(--border);
      z-index:50;
    }
    .logo{font-weight:700; letter-spacing:.08em; text-transform:uppercase; font-size:.9rem;}
    nav a{mar

In [6]:
from IPython.display import HTML

with open('listings.html', 'r') as f:
    html_content = f.read()

display(HTML(html_content))